In [3]:
!pip3 install transformers

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from transformers import AutoModel, AutoTokenizer

In [5]:
# Define paths
positive_data_path = "/content/turkey_052020_tweets_csv_hashed_2011.csv"
negative_data_path = "/content/turkey_052020_tweets_csv_hashed_2020_01.csv"
model_path = "dbmdz/bert-base-turkish-cased"

In [6]:
positive_raw_data = pd.read_csv(positive_data_path)
positive_raw_data["label"] = 1.0

negative_raw_data = pd.read_csv(negative_data_path)
negative_raw_data["label"] = 0.0

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
positive_raw_data.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_userid',
       'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions', 'label'],
      dtype='object')

In [8]:
positive_raw_data.shape

(35100, 31)

In [9]:
negative_raw_data.shape

(859516, 31)

In [10]:
merged_data = pd.concat([positive_raw_data, negative_raw_data])
merged_data = merged_data.sample(frac=1)

In [11]:
merged_data

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_userid,in_reply_to_tweetid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,label
209450,1218277236144529410,ouUEEycIDom7hjMLJDaoYdcG5yZBuWiLBI3BmSiYw=,ouUEEycIDom7hjMLJDaoYdcG5yZBuWiLBI3BmSiYw=,ouUEEycIDom7hjMLJDaoYdcG5yZBuWiLBI3BmSiYw=,NaN,NaN,NaN,203,433,2019-12-22,en,tr,RT @OkanKars3: Çok kişiyi özgürlüğüne kavuş...,2020-01-17 21:01,Twitter for iPhone,NaN,NaN,NaN,True,NaN,1.218263e+18,absent,absent,0,0,0,0,['AdliMahkumKapsamlıEşitYasa'],[],['1064868786002714625'],0.0
41693,1212640138267910144,oq3N71ou4gro4s5VZ4Je1RzLz7iNxp5y5AbeilSz4=,oq3N71ou4gro4s5VZ4Je1RzLz7iNxp5y5AbeilSz4=,oq3N71ou4gro4s5VZ4Je1RzLz7iNxp5y5AbeilSz4=,NaN,NaN,NaN,15,18,2019-07-05,tr,tr,@TurkHaberAjansi @BirGun_Gazetesi @gazeteduvar...,2020-01-02 07:41,Twitter Web App,oq3N71ou4gro4s5VZ4Je1RzLz7iNxp5y5AbeilSz4=,1.212640e+18,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],[],"['723236478801842176', '99620272', '7434051482...",0.0
215575,1212872028493144064,iu3Xfy2L+dLddWhegk4jjrVp3PUQ23ZgBU3TqdzAtjo=,iu3Xfy2L+dLddWhegk4jjrVp3PUQ23ZgBU3TqdzAtjo=,iu3Xfy2L+dLddWhegk4jjrVp3PUQ23ZgBU3TqdzAtjo=,NaN,Ölümüne Ölümüne RTE,NaN,1102,723,2018-06-08,tr,tr,"RT @Aydnmermerci: Batılının, silah zoruyla, at...",2020-01-02 23:03,Twitter for Android,NaN,NaN,NaN,True,NaN,1.212601e+18,absent,absent,0,0,0,0,[],[],['1084391797382627329'],0.0
261314,1219550818023739392,7f0RTlbCQBhzkFRwVcaXRxRKLnAoabvH9pHNUijTI=,7f0RTlbCQBhzkFRwVcaXRxRKLnAoabvH9pHNUijTI=,7f0RTlbCQBhzkFRwVcaXRxRKLnAoabvH9pHNUijTI=,NaN,NaN,NaN,448,532,2017-04-10,tr,tr,RT @sevilmis01: “...\n\nnefsinin isteklerine (...,2020-01-21 09:22,Twitter for Android,NaN,NaN,NaN,True,NaN,1.219533e+18,absent,absent,0,0,0,0,[],[],['588419294'],0.0
87306,1213416440856006656,yLePHFMCDbnnskNzJsiZzGuyMPvM7ly9vH5QyWkslc=,yLePHFMCDbnnskNzJsiZzGuyMPvM7ly9vH5QyWkslc=,yLePHFMCDbnnskNzJsiZzGuyMPvM7ly9vH5QyWkslc=,Antalya,VUK 359 bizi bitirdi. Artık düzeltilsin.,NaN,198,40,2013-06-17,tr,tr,"""Vergi barışı"" dediniz, yaptık. ""Öde"" dediniz ...",2020-01-04 11:06,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],['https://twitter.com/stayfre32254781/status/1...,"['3160777371', '70363441']",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313304,1216646745179947009,F2AiYU+QSGcZsOeAsALogQJ3J6lePgDOlh138RKN2Hs=,F2AiYU+QSGcZsOeAsALogQJ3J6lePgDOlh138RKN2Hs=,F2AiYU+QSGcZsOeAsALogQJ3J6lePgDOlh138RKN2Hs=,"Bağcılar, İstanbul",Aşkım İçin Herşeyden Vaz geçerim !! Özgürlüğüm...,https://t.co/2YwxINzJHW,687,204,2013-06-13,tr,tr,"Adalet nerede hesap sorarsa, merhamet orada ha...",2020-01-13 09:02,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],[],['304611571'],0.0
542405,1216121215737307136,+THAMZXhqpRBnR11ouT0qOaB1e5sKEaMZRCnAFHsVzk=,+THAMZXhqpRBnR11ouT0qOaB1e5sKEaMZRCnAFHsVzk=,+THAMZXhqpRBnR11ouT0qOaB1e5sKEaMZRCnAFHsVzk=,NaN,NaN,NaN,4070,3986,2019-05-12,en,tr,RT @DewPwbPAqc6J2UZHeHLDGGubsrmjT8a2YxsG14m8lI...,2020-01-11 22:14,Twitter for iPhone,NaN,NaN,NaN,True,DewPwbPAqc6J2UZHeHLDGGubsrmjT8a2YxsG14m8lIo=,1.216121e+18,absent,absent,0,0,0,0,['EytUmutDirenmektirDiyor'],[],['DewPwbPAqc6J2UZHeHLDGGubsrmjT8a2YxsG14m8lIo='],0.0
810049,1215993800641929217,h2w6lRTjEQSNf4Kbbsdj6+U+DWiD2l3+2FMti38OWI=,h2w6lRTjEQSNf4Kbbsdj6+U+DWiD2l3+2FMti38OWI=,h2w6lRTjEQSNf4Kbbsdj6+U+DWiD2l3+2FMti38OWI=,NaN,NaN,NaN,495,294,2019-11-21,tr,tr,"Verilen söz, vaktinde verilmesi gereken bir bo...",2020-01-11 13:48,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,1,2,[],[],['79574414'],0.0
763729,1220060559854526475,pcFZ7NxBN3OgtA067v5AsOHQTuzMIwc98q6sXT5N3lg=,pcFZ7NxBN3OgtA067v5AsOHQTuzMIwc98q6sXT5N3lg=,pcFZ

In [12]:
train_data = merged_data[:800000]
test_data = merged_data[800000:]

In [13]:
# Feature info
float_features_names = [
  "follower_count",
  "following_count",
  "is_retweet",
  "quote_count",
  "reply_count",
  "like_count",
  "retweet_count",
]

In [14]:
# Data pre-processing
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [21]:
def process_data(batch):
  feature_dict = tokenizer(
      batch["tweet_text"].tolist(),
      padding=True,
      truncation=True,
      return_tensors="pt",
  )

  numerical_data = batch[float_features_names].astype(float)
  feature_dict["float_features"] = torch.from_numpy(numerical_data.values).float()

  labels = batch["label"].astype(float)
  feature_dict["labels"] = torch.from_numpy(labels.values).float()

  return feature_dict

In [16]:
class BertTweetClassifier(nn.Module):
  def __init__(self, hidden_size, dense_size, numeric_feature_size, output_size, dropout=0.1):
    super().__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size

    # Use pre-trained BERT model
    self.bert = AutoModel.from_pretrained(
        model_path,
        output_hidden_states=True,
        output_attentions=True,
    )

    for param in self.bert.parameters():
        param.requires_grad = False  # No backprop here for now

    self.weights = nn.Parameter(torch.rand(13, 1))
    self.dropout = nn.Dropout(dropout)
    self.fc1 = nn.Linear(hidden_size, dense_size)
    self.fc2 = nn.Linear(dense_size + numeric_feature_size, output_size)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, feature_dict):
    input_ids = feature_dict["input_ids"]
    float_features = feature_dict["float_features"]

    all_hidden_states, all_attentions = self.bert(input_ids)[-2:]
    batch_size = input_ids.shape[0]

    ht_cls = torch.cat(all_hidden_states)[:, :1, :].view(13, batch_size, 1, self.hidden_size)
    atten = torch.sum(ht_cls * self.weights.view(13, 1, 1, 1), dim=[1, 3])
    atten = F.softmax(atten.view(-1), dim=0)
    feature = torch.sum(ht_cls * atten.view(13, 1, 1, 1), dim=[0, 2])

    dense_out = self.fc1(self.dropout(feature))
    activ_out = self.relu(dense_out)
    concat_layer = torch.cat((activ_out, float_features), 1)
    out = self.fc2(concat_layer)
    prediction = self.sigmoid(out)

    return prediction

In [17]:
model = BertTweetClassifier(768, 32, 7, 1)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
model

BertTweetClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [24]:
batch_size = 32
# num_examples = train_data.shape[0]
num_examples = 100
criterion = nn.BCELoss()

for i in range(0, num_examples, batch_size):
  batch_data = train_data[i:min(i+batch_size, num_examples)]
  feature_dict = process_data(batch_data)
  predictions = np.squeeze(model(feature_dict))
  labels = feature_dict["labels"]
  loss = criterion(predictions, labels)
  print(loss)

tensor(85.2968, grad_fn=<BinaryCrossEntropyBackward>)
tensor(91.5606, grad_fn=<BinaryCrossEntropyBackward>)
tensor(82.0463, grad_fn=<BinaryCrossEntropyBackward>)
tensor(100., grad_fn=<BinaryCrossEntropyBackward>)
